In [24]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.cluster import KMeans
import numpy as np

In [32]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)


Tutaj metodologia jest zupełnie inna ponieważ QKeras nie obsługuje kwantyzacji za pomocą metody centroidów i musiałem zrobić własną implementację

In [26]:
def quantize_weights(model, n_clusters=16):
    for layer in model.layers:
        if 'dense' in layer.name:
            weights, biases = layer.get_weights()
            # Kwantyzacja wag
            kmeans = KMeans(n_clusters=n_clusters, n_init=10)
            flat_weights = weights.reshape((-1, 1))
            kmeans.fit(flat_weights)
            quantized_flat_weights = kmeans.cluster_centers_[kmeans.predict(flat_weights)]
            quantized_weights = quantized_flat_weights.reshape(weights.shape)
            # Ustawianie kwantyzowanych wag
            layer.set_weights([quantized_weights, biases])

In [34]:
from sklearn.model_selection import train_test_split
acc = np.zeros((8, 5))
for j in range(0, 5):
    X_full = np.concatenate((X_train, X_test))
    y_full = np.concatenate((y_train, y_test))

    X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(200, activation='sigmoid'),
        Dense(80, activation='sigmoid'),
        Dense(10, activation='sigmoid')
    ])

    for i in range(0, 8):
        print(f"Pula {j+1}, kwantyzacja INT{i+1}")
        q_model = model
        q_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        q_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
        quantize_weights(q_model, 2**(i+1))
        acc[i, j] = q_model.evaluate(X_test, y_test)[1]
        

(56000, 28, 28) (14000, 28, 28) (56000, 10) (14000, 10)
Pula 0, kwantyzacja INT1
Epoch 1/3
1750/1750 [==============================] - 5s 3ms/step - loss: 0.5225 - accuracy: 0.8638 - val_loss: 0.3271 - val_accuracy: 0.9049
Epoch 2/3
1750/1750 [==============================] - 5s 3ms/step - loss: 0.3116 - accuracy: 0.9076 - val_loss: 0.2915 - val_accuracy: 0.9146
Epoch 3/3
438/438 [==============================] - 1s 1ms/step - loss: 0.7503 - accuracy: 0.7605
Pula 0, kwantyzacja INT2
Epoch 1/3
1750/1750 [==============================] - 5s 3ms/step - loss: 0.3056 - accuracy: 0.9098 - val_loss: 0.2709 - val_accuracy: 0.9186
Epoch 2/3
1750/1750 [==============================] - 5s 3ms/step - loss: 0.2894 - accuracy: 0.9115 - val_loss: 0.2592 - val_accuracy: 0.9216
Epoch 3/3
1749/1750 [============================>.] - ETA: 0s - loss: 0.2658 - accuracy: 0.9165